# TODOs (project)

- Add NA values (see "data.archive.yaml")
- Add lab_ids for material sources
- Add date when we have change ISO format (see https://github.com/Automate-Solar/BERTHA/pull/11)


In [1]:
import json
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
# from Nomad_API import *
import yaml
# import zipfile

In [3]:
with open('/home/hampusnasstrom/Downloads/data.json') as f:
    data = json.load(f)

In [4]:
dic = data[0]

In [5]:
dic

{'_id': {'$oid': '661f6bb43e6a8ca8db8a841e'},
 'datetime': {'$date': '2024-04-17T08:26:44.937Z'},
 'Campaign code': 'BaZrSn_002',
 'Campaign description': 'Sputter setup with Ba, Zr and Sn for development of first ML algorithms',
 'Material Ax1': 'Ba',
 'Material Ax2': 'Zr',
 'Material Ax3': 'Sn',
 'Magnetron Ax1': 5,
 'Magnetron Ax2': 1,
 'Magnetron Ax3': 3,
 'Power Supply Ax1': 5,
 'Power Supply Ax2': 2,
 'Power Supply Ax3': 3,
 'Power Supply Ax1 Ramp Rate [W/s]': 5,
 'Power Supply Ax2 Ramp Rate [W/s]': 10,
 'Power Supply Ax3 Ramp Rate [W/s]': 10,
 'Power Supply Ax1 Pulse Rate [kHz or RF]': 'RF',
 'Power Supply Ax2 Pulse Rate [kHz or RF]': '50',
 'Power Supply Ax3 Pulse Rate [kHz or RF]': '50',
 'Sputter gas': 'Ar',
 'QCM_crystal_type': 'KJLC LI008010G10 6MHz Gold',
 'Series_start_date/time': '2024-04-17 08:23:11.139518',
 'Base_pressure_(start)_[Torr]': '4.1913154391E-8',
 'Operator_name': '',
 'Series_ID': 'BaZrSn_002_006',
 'Series_description': '15x15 grid of power pressure combi

In [ ]:
# print(yaml.dump(steps))

In [6]:
def getPoweredAxes(arr):
    axes = []
    for i in range(len(arr)):
        if arr[i]:
            axes.append(str(i+1))
    return axes

In [8]:
def getYamlMagnetrons(dic, power_axes):
    axes = getPoweredAxes(power_axes)
    return [getYamlMagnetron(dic, axis) for axis in axes]

In [18]:
def getYamlMagnetron(dic, axis):
    magnetron = {
        'name' : dic['Material Ax' + axis],
        'material' : {
            'name' : dic['Material Ax' + axis],
            'lab_id' : None
        },
        'vapor_source' : {
            'setpoints' : {
            'set_power' : dic['Power_Ax' + axis + '_setpoint_[W]'],
            'set_voltage' : None,
            'set_current' : None
            },
            'power' : {
                'values' : dic['Actual_Power_Ax' + axis + '_[W]'],
                'times' : dic['Time_[s]'],
                'mean' : dic['Actual_Power_Ax' + axis + '_[W]_Mean'],
                'error' : dic['Actual_Power_Ax' + axis + '_[W]_STD']
            },
            'voltage' : {
                'values' : dic['Voltage_Ax' + axis + '_[V]'],
                'times' : dic['Time_[s]'],
                'mean' : dic['Voltage_Ax' + axis + '_[V]_Mean'],
                'error' : dic['Voltage_Ax' + axis + '_[V]_STD'],
            },
            'current' : {
                'values' : [],
                'times' : dic['Time_[s]'],
                'mean' : None,
                'error' : None
            },
            'power_supply' : {
                'instrument_id' : "Power Supply Ax" + axis,
                'frequency' : None
            }
        },
        'position' : {
            'center_xyz' : None,
            'center_normal' : None,
            'rotation' : None
        }
    }
    return magnetron

In [9]:
arr = [True, True, True]

        
#getYamlMagnetrons(dic, arr)
        

# Environment

In [10]:
def getYamlQCM(dictionary, i):
    prefix = 'QCM_' + str(i) + '_'
    yamlQCM = {
            'crystal_info' : {
                'info' : None,
                'resonant_frequency' : None
            },
            'sensor_data' : {
                'values' : dictionary[prefix + 'frequency_[Hz]'],
                'mean' : None,
                'error' : None,
                'slope' : dictionary[prefix + 'rate_[df/dt]']
            },
            'remaining_lifetime' : {
                'value' : dictionary[prefix + 'xtal_lifetime_[%]']
            },
            'mass_deposition_rate' : {
                'value' : None,
                'error' : None
            }
        }
    return yamlQCM

In [11]:
def getYamlEnv(dictionary):
    environment = {
        'gas_flow' : {
            'values' :  dictionary['Gas_flow_[sccm]'],
            'mean' : dictionary['Gas_flow_[sccm]_Mean'],
            'error' : dictionary['Gas_flow_[sccm]_STD'],
            'gas' : 'Here we do the pubchem thing'
        },
        'setpoints' : {
            'pressure' : dictionary['Pressure_setpoint_[mTorr]'],
            'flow' : None,
        },
        'pressure' : {
            'values' : dictionary['Actual_pressure_[mTorr]'],
            'mean' : dictionary['Actual_pressure_[mTorr]_Mean'],
            'error' : dictionary['Actual_pressure_[mTorr]_STD']
        },
        'sensors' : [getYamlQCM(dictionary, i+1) for i in range(3)]
    }
    return environment
    


In [ ]:
#environment = getYamlEnv(dic)

In [19]:
data = {'data' : {
            'm_def' : "../uploads/VqPwTxNlScSdvgUOcF21Ag/raw/automate-solar_schema_latest.archive.yaml#/definitions/section_definitions/AutomateSolarSputterDeposition",
            'quantity' : None,
            'name' : dic['Run_ID'],
            'lab_id' : "Uppsala University Åutomate-Solar",
            'description' : f'{dic["Campaign description"]}, {dic["Campaign code"]}, {dic["Series_description"]}',
            'steps' : [
                {
                    'sources' : getYamlMagnetrons(dic, [True,True,True]),
                    'environment' : getYamlEnv(dic)
                }
            ]
    }
}

In [14]:
print(yaml.dump(data, default_flow_style=False))

data:
  description: Sputter setup with Ba, Zr and Sn for development of first ML algorithms,
    BaZrSn_002, 15x15 grid of power pressure combinations for Sn
  lab_id: "Uppsala University \xC5utomate-Solar"
  m_def: ../uploads/5Vhw2cHTQZW68lGJ1V_7QA/raw/automate-solar_schema.archive.yaml#/definitions/section_definitions/AutomateSolarSputterDeposition
  name: BaZrSn_002_006_1
  quantity: null
  steps:
  - environment:
      gas_flow:
        error: 0.223
        gas: Here we do the pubchem thing
        mean: 42.146
        values:
        - 0.594
        - 0.594
        - 0.594
        - 0.216
        - 0.216
        - 0.119
        - 0.119
        - 0.119
        - 1.073
        - 1.073
        - 22.215
        - 22.215
        - 30.788
        - 30.788
        - 36.074
        - 36.074
        - 36.074
        - 36.074
        - 37.962
        - 38.424
        - 38.424
        - 39.264
        - 39.264
        - 40.774
        - 40.774
        - 40.774
        - 42.153
        - 42.

In [20]:
with open('data_test.archive.yml', 'w') as outfile:
    yaml.dump(data, outfile, default_flow_style=False)

In [ ]:
#with zipfile.ZipFile('data.zip', 'w') as zipped_f:
    #zipped_f.writestr("data.archive.yaml", yaml.dump(data, default_flow_style=False))

In [ ]:
#test = yaml.dump(data, default_flow_style=False)

In [ ]:
{['test'] : ['test']}

TypeError: unhashable type: 'list'

# Source 1,2,3 repeats in an array: [source1, source2, source3]

magnetron1 = {
    'name' : dic['Material Ax1'],
    'material' : {
        'name' : dic['Material Ax1'],
        'lab_id' : None
    },
    'vapor_source' : {
        'setpoints' : {
        'set_power' : dic['Power_Ax1_setpoint_[W]'],
        'set_voltage' : None,
        'set_current' : None
        },
        'power' : {
            'values' : dic['Actual_Power_Ax1_[W]'],
            'times' : dic['Time_[s]'],
            'mean' : dic['Actual_Power_Ax1_[W]_Mean'],
            'error' : dic['Actual_Power_Ax1_[W]_STD']
        },
        'voltage' : {
            'values' : dic['Voltage_Ax1_[V]'],
            'times' : dic['Time_[s]'],
            'mean' : dic['Voltage_Ax1_[V]_Mean'],
            'error' : dic['Voltage_Ax1_[V]_STD'],
        },
        'current' : {
            'values' : None,
            'times' : dic['Time_[s]'],
            'mean' : None,
            'error' : None
        },
        'power_supply' : {
            'instrument_id' : "Power Supply Ax1",
            'frequency' : dic['Power Supply Ax1 Pulse Rate [kHz or RF]']
        }
    },
    'position' : {
        'center_xyz' : None,
        'center_normal' : None,
        'rotation' : None
    }
}



steps = [magnetron1] # Append for source 2,3